# Imports

In [ ]:
def fix_layout(width:int=95):
    from IPython.core.display import display, HTML
    display(HTML('<style>.container { width:' + str(width) + '% !important; }</style>'))
    
fix_layout()

In [ ]:
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

import nnpde.functions.iterative_methods as im
from nnpde.functions import geometries, helpers
from nnpde.utils.logs import enable_logging, logging 
from nnpde.problems import DirichletProblem 
from nnpde.utils import plots

In [ ]:
enable_logging(20)

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Setup

In [ ]:
# Define train dimension: NxN
N = 16

# For each problem instance define number of iteration to perform to obtain the solution
nb_problem_instances = 30
problem_instances = [DirichletProblem(k=k) for k in np.random.randint(1, 20, nb_problem_instances)]

# Hyper-parameter search learning rate

In [ ]:
import nnpde.model as M 
reload(M)

nb_layers = 3
max_epochs = 100
batch_size = 10
losses = []

# SGD
learning_rates = np.logspace(start=-6, stop=-4, num=7) #num=7 is good since it contains 1e-5
hyper_models = [M.JacobyWithConv(max_epochs=max_epochs, batch_size=batch_size, learning_rate=learning_rate, nb_layers=nb_layers)
          for learning_rate in learning_rates]
#Adadelta
hyper_models.append(M.JacobyWithConv(max_epochs=max_epochs, batch_size=batch_size, optimizer="Adadelta", nb_layers=nb_layers))
  
for model in hyper_models:
    model.fit(problem_instances)

In [ ]:
# Colors for plotting
color_map = plt.get_cmap('cubehelix')
colors = color_map(np.linspace(0.1, 1, 10))

# Initilize figure
hyper_fig = plt.figure()

# Plot SGD
i = 0
for model in hyper_models[:-1]:  
    n_epoch = np.arange(np.shape(model.losses)[0])
    plt.semilogy(n_epoch, model.losses, color=colors[i], linewidth=1, linestyle="-", marker=(i+2, 0, 0), markevery=10, label = '$\gamma= {0:.2e} $'.format(learning_rates[i]))
    i += 1

# Plot Adadelta
n_epoch = np.arange(np.shape(hyper_model[-1].losses)[0])
plt.semilogy(n_epoch, hyper_model[-1].losses, color=colors[i], linewidth=1, linestyle="-", marker=(i+2, 0, 0), markevery=10,label='Adadelta')

# Additional settings
plt.legend(bbox_to_anchor=(1.05, 0.31), loc=3, borderaxespad=0.)
plt.xlabel('n epochs', fontsize=14)
plt.ylabel('Total loss [-]', fontsize=14)
plt.xlim([0, max_epochs])
plt.ylim([0, 800])
plt.title('Loss evolution for different learning rates, $K=3$, batchSize=10')
plt.grid(True, which = "both", linewidth = 0.5,  linestyle = "--")

#hyper_fig.savefig('../report/fig/hyper.eps', bbox_inches='tight')
plt.draw()
plt.show()

# Train model using K = 1,2,3,4,5

In [ ]:
max_epochs = 100
batch_size = 10
models = [M.JacobyWithConv(max_epochs=max_epochs, batch_size=batch_size, optimizer="Adadelta", nb_layers=nb_layers)
          for nb_layers in [1,2,3,4,5]]

for model in models:
    model.fit(problem_instances)

In [ ]:
# Colors for plotting
color_map = plt.get_cmap('cubehelix')
colors = color_map(np.linspace(0.1, 1, 10))

# Initilize figure
comparison_K_fig = plt.figure()

# Plot SGD
i = 0
for model in models[:]:  
    n_epoch = np.arange(np.shape(model.losses)[0])
    plt.semilogy(n_epoch, model.losses, color=colors[i], linewidth=1, linestyle="-", marker=(i+2, 0, 0), markevery=10, label = '$K= {0} $'.format(model.nb_layers))
    print("For K={0} final loss is {1}".format(model.nb_layers, model.losses[-1]))
    i += 1

# Additional settings
plt.legend(bbox_to_anchor=(1.05, 0.31), loc=3, borderaxespad=0.)
plt.xlabel('n epochs', fontsize=14)
plt.ylabel('Total loss [-]', fontsize=14)
plt.xlim([0, max_epochs])
#plt.ylim([0, 800])
plt.title('Loss evolution for different learning rates, $K=3$, batchSize=10')
plt.grid(True, which = "both", linewidth = 0.5,  linestyle = "--")

#hyper_fig.savefig('../report/fig/comparison_K.eps', bbox_inches='tight')
plt.draw()
plt.show()

# Train using the class

# Test on a bigger grid

In [ ]:
from nnpde.metrics import least_squares_loss as LSE

In [ ]:
N = 50
nb_iters = 2000
net = models[4].net

B_idx, B = geometries.square_geometry(N)

# Set forcing term
f = torch.ones(1,1,N,N)*1.0

# Obtain solutions
gtt = im.jacobi_method(B_idx, B, f, torch.ones(1,1,N,N), k = 10000)
jacoby_pure = im.jacobi_method(B_idx, B, f, torch.ones(1,1,N,N), k = nb_iters)
output = im.H_method(net, B_idx, B, f, torch.ones(1,1,N,N), k = nb_iters)

print(f"error after {nb_iters} for both methods: {LSE(output, gtt)}, and ground truth and jacoby {LSE(gtt, jacoby_pure)}")

In [ ]:
loss_to_be_achieved = 1e-3

u_0 = torch.ones(1, 1, N, N)
net = model.net

In [ ]:
%%timeit

u_k_old = im.jacobi_method(B_idx, B, f, u_0, k = 1)
loss_of_old = LSE(gtt, u_k_old)
k_count_old = 1
count_old = 1
# old method 
while loss_of_old >= loss_to_be_achieved:
    u_k_old = im.jacobi_method(B_idx, B, f, u_k_old, k = 1)
    loss_of_old = LSE(gtt, u_k_old)
    k_count_old += 1
    
print(k_count_old)

In [ ]:
%%timeit

u_k_new = im.H_method(net, B_idx, B, f, u_0, k=1)

loss_new = LSE(gtt, u_k_new)
k_count_new = 1


# new method

while loss_new >= loss_to_be_achieved:
    u_k_new = im.H_method(net, B_idx, B, f, u_k_new, k=1)
    loss_new = LSE(gtt, u_k_new)
    k_count_new += 1
    
print(k_count_new)

In [ ]:
# This is not correct, but we have to look for a way to access the variables inside timeit

print("needed {0} iterations (compared to {1}), ratio: {2}".format(k_count_old, k_count_new, k_count_old/k_count_new))

In [ ]:
print("the loss of the new method is {0}, compared to the pure-jacoby one: {1}. computed with {2} iterations".format(F.mse_loss(gtt, output), F.mse_loss(gtt, jacoby_pure), nb_iters))

In [ ]:
helpers.plot_solution(gtt,output,N)

In [ ]:
(gtt.view(N,N) - output.view(N,N)).mean()

Test on L-shape domain

In [ ]:
B_idx, B = geometries.l_shaped_geometry(N)

# Set forcing term
f = torch.ones(1,1,N,N)*1.0

# Obtain solutions
gtt = im.jacobi_method(B_idx, B, f, torch.ones(1,1,N,N), k = 10000)
output = im.H_method(net, B_idx, B, f, torch.ones(1,1,N,N), k = 2000)

In [ ]:
helpers.plot_solution(gtt,output,N)

In [ ]:
compare_flops(16,k_count_new,k_count_old,3)

In [ ]:
Spectral radius. Don't remove Francesco will delete me

In [ ]:
B_idx = problem_instances[1].B_idx
net = nn.Sequential(nn.Conv2d(1, 1, 3, padding=1, bias=False))
G = helpers.build_G(B_idx)
T = helpers.build_T(N)
H = helpers.conv_net_to_matrix(net, N)
I = np.eye(N)
helpers.spectral_radius(T+G.dot(H).dot(T)-G.dot(H))